## Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
local_number = 1
receive_numbers = [2,3,4]
iteration_per_communicate = 1
add = '/content/drive/MyDrive/update/'

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import csv
import time
import random
import matplotlib.image as img
# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')

import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import RandomSampler
from torch.utils.data import TensorDataset
from torchvision.utils import make_grid
import torchvision.models as models
import time
from PIL import Image
from tqdm import tqdm
import gc
from torch.utils.data import DataLoader
# from torchsummary import summary

## Import Dataset

In [ ]:
class Musicdata(torch.utils.data.Dataset):
    def __init__(self, npz, mode='train'):
        npzfile = np.load(npz)
        self.mode = mode   
        self.x = npzfile['arr_0']
        self.x = [one.reshape(1,640,128) for one in self.x]
        self.y = npzfile['arr_1']

        
                    
    def __getitem__(self, index):
        data = torch.tensor(self.x[index], dtype = torch.float32)
        #data = transforms(data)
        if self.mode == 'test': 
            return data
        genre = [np.where(one == 1)[0][0] for one in self.y]
        genre = torch.tensor(int(genre[index]))
        return data, genre

    def __len__(self):
        return len(self.x)

In [ ]:
dataset_train = Musicdata('/content/drive/MyDrive/shuffled_train.npz', mode='train')
dataset_val = Musicdata('/content/drive/My Drive/shuffled_valid.npz', mode='val')
# dataset_test = Musicdata('/content/drive/My Drive/test_arr.npz', mode='test')
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset_train, batch_size=32, shuffle=False, num_workers = 8)
val_loader = DataLoader(dataset_val, batch_size=32, shuffle=False, num_workers = 8)
# test_loader = DataLoader(dataset_test, batch_size=128, shuffle=False)

## Building Model Architecture

## CNN(ResNet-based) + LSTM
The following architecture is based on ResNet.

### Define `Conv1` layer and `Bottleneck` class

In [ ]:
def Conv1(in_planes, places, stride=2):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_planes, out_channels=places, kernel_size=7, stride=stride, padding=3, bias=False),
        nn.BatchNorm2d(places),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

class Bottleneck(nn.Module):
  '''
  in_places: size of input channel
  places: 進行前兩次捲機時的 output channel size
  stride: 第二次卷機時的 stride
  expansion: bottleneck 輸出的 channel 為 places*expansion
  downsampling: 是否要改變 H, W
  '''
  def __init__(self,in_places,places, stride=1, downsampling=False, expansion = 4):
        super(Bottleneck,self).__init__()
        self.expansion = expansion
        self.downsampling = downsampling

        self.bottleneck = nn.Sequential(
            # 第一次是為了改變 channel 的維度，H, W 不變
            nn.Conv2d(in_channels=in_places,out_channels=places,kernel_size=1,stride=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            # 第二次是是做捲機，H, W 會變
            nn.Conv2d(in_channels=places, out_channels=places, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            # 第三次是為了改變 channel 的維度為 places*self.expansion，H, W 不變
            nn.Conv2d(in_channels=places, out_channels=places*self.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(places*self.expansion),
        )
        

        if self.downsampling:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels=in_places, out_channels=places*self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(places*self.expansion)
            )
        self.relu = nn.ReLU(inplace=True)
  def forward(self, x):
        residual = x
        out = self.bottleneck(x)

        if self.downsampling:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out

### Define `ResNet` class

In [ ]:
class RESTM(nn.Module):
    def __init__(self, blocks, initial_size, expansion=4, hidden_size=32, num_layers=2, fc_neural=512, fc_block=1):
        super(RESTM,self).__init__()
        self.expansion = expansion
        self.initial_size = initial_size

        self.conv1 = Conv1(in_planes=1, places=self.initial_size)
        set_size = self.initial_size
        self.layer1 = self.make_layer(in_places=set_size, places=set_size, block=blocks[0], stride=1, expansion=self.expansion)
        set_size *= self.expansion
        self.layer2 = self.make_layer(in_places=set_size, places=int(set_size/2), block=blocks[1], stride=2, expansion=self.expansion)
        set_size *= self.expansion
        self.layer3 = self.make_layer(in_places=int(set_size/2), places=int(set_size/4), block=blocks[2], stride=2, expansion=self.expansion)
        set_size *= self.expansion
        self.layer4 = self.make_layer(in_places=int(set_size/4), places=int(set_size/8), block=blocks[3], stride=2, expansion=self.expansion)
        set_size *= self.expansion

        self.avgpool = nn.AvgPool2d(2, stride=1)
        self.LSTM = nn.LSTM(int(set_size/8), hidden_size, num_layers, batch_first = True)
        
        self.fc = self.make_fc(hidden_size=hidden_size, fc_neural=fc_neural, fc_block=fc_block)
#         self.fc = nn.Sequential(
#                 nn.Dropout(),
#                 nn.Linear(3*19*hidden_size, fc_neural),
#                 nn.BatchNorm1d(fc_neural),
#                 nn.ReLU(True),
#                 nn.Dropout(),

#                 nn.Linear(fc_neural, 8),
#         )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def make_layer(self, in_places, places, block, stride, expansion):
        layers = []
        layers.append(Bottleneck(in_places, places, stride, downsampling=True, expansion=expansion))
        for i in range(1, block):
            layers.append(Bottleneck(places*self.expansion, places, expansion=expansion))

        return nn.Sequential(*layers)
    
    def make_fc(self, hidden_size, fc_neural, fc_block):
        layers = [nn.Dropout(), nn.Linear(3*19*hidden_size, fc_neural), nn.BatchNorm1d(fc_neural), nn.ReLU(True), nn.Dropout()]
        for i in range(fc_block):
            layers += [nn.Linear(fc_neural, fc_neural), nn.BatchNorm1d(fc_neural), nn.ReLU(True), nn.Dropout()]
        layers += [nn.Linear(fc_neural, 8)]
        
        return nn.Sequential(*layers)


    def forward(self, x):
#         size before everything: bach size * 1 * 640 * 128
#         size after conv1:       bach size * initial size * 160 * 32
#         size after layer1:      bach size * setting      * 160 * 32
#         size after layer2:      bach size * setting      *  80 * 16
#         size after layer3:      bach size * setting      *  40 *  8
#         size after layer4:      bach size * setting      *  20 *  4
#         size after avgpooling:  bach size * setting      *  19 *  3
#         size after view:        bach size * 57 * setting
#         size after LSTM:        bach size * (57 * setting)

#         print("size before everything:", x.shape)
        x = self.conv1(x)
#         print("size after conv1", x.shape)
        x = self.layer1(x)
#         print("size after layer1", x.shape)
        x = self.layer2(x)
#         print("size after layer2",x.shape)
        x = self.layer3(x)
#         print("size after layer3",x.shape)
        x = self.layer4(x)
#         print("size after layer4",x.shape)
        x = self.avgpool(x)
#         print("size after average pooling",x.shape)
        #x = x.view(x.size(0), -1)
        x = x.view(x.size(0), int(3*19), -1)
#         print("size after view", x.shape)
        x, _ = self.LSTM(x)
        x = x.reshape(x.size(0), -1)
#         print("size after LSTM", x.shape)
        x = self.fc(x)
        return x

# model = ResNet([1, 2, 1, 1], initial_size=64)
# summary(model, (1,128,640))

### Define `configuration` function

In [ ]:
def configuration(initial_size=64, blocks_1=1, blocks_2=1, blocks_3=1, blocks_4=1, expansion=4, hidden_size=32, num_layers=2, fc_neural=512, fc_block=1):
    block = [blocks_1, blocks_2, blocks_3, blocks_4]
    return RESTM(block, initial_size, expansion, hidden_size, num_layers, fc_neural, fc_block)

# model = configuration(1, 2, 3, 4, 5, 6, 7)
# summary(model, (1,128,640))

## Criterion

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()

In [ ]:
def train(input_data, model, criterion, optimizer, output_disable = False):

    model.train()
    loss_list = []
    total_count = 0
    acc_count = 0
    pbar = tqdm(input_data, position = 0)
    for data in pbar:
        pbar.set_description("Training")
        music, genre = data[0].cuda(), data[1].cuda()
        optimizer.zero_grad()
        outputs = model(music)
        loss = criterion(outputs, genre) ##data type
        loss.backward()
        optimizer.step() 
        _, predicted = torch.max(outputs.data, 1)
        total_count += genre.size(0) 
        acc_count += (predicted == genre).sum()  
        loss_list.append(loss.item())
        

    acc = acc_count.to("cpu").detach().numpy() / total_count
    loss = sum(loss_list) / len(loss_list)
    return acc, loss

In [ ]:
def val(input_data, model, criterion, output_disable = False):
    model.eval()
    
    loss_list = []
    total_count = 0
    acc_count = 0
    pbar = tqdm(input_data, position = 0)
    with torch.no_grad():
        for data in pbar:
            pbar.set_description("Validation")
            music, genre = data[0].cuda(), data[1].cuda()
            outputs = model(music)
            loss = criterion(outputs, genre) ##data type
            _, predicted = torch.max(outputs.data, dim=1)
            total_count += genre.size(0)
            acc_count += (predicted == genre).sum()
            loss_list.append(loss.item())


    acc = acc_count.to("cpu").detach().numpy() / total_count
    loss = sum(loss_list) / len(loss_list)
    return acc, loss

## CNN(ResNet-based) + LSTM Training Process
The following architecture is based on ResNet.

In [ ]:
def RESTM_Train(parameters ,max_epochs = 3):    
    learning_rate = parameters[0]
    initial_size, blocks_1, blocks_2, blocks_3, blocks_4, expansion, hidden_size, num_layers, fc_neural, fc_block = tuple(int(param) for param in parameters[1:])
    
    print(" learning_rate =", learning_rate, "initial_size =", initial_size, "\n",\
          "blocks_1 =",blocks_1, "blocks_2 =",blocks_2, "blocks_3 =",blocks_3, "blocks_4 =",blocks_4, "\n", \
          "expansion =", expansion, "hidden_size =", hidden_size, "num_layers =", num_layers)
    
    train_acc_list = []
    train_loss_list = []
    val_acc_list = []
    val_loss_list = []
    
    model = configuration(initial_size, blocks_1, blocks_2, blocks_3, blocks_4, expansion, hidden_size, num_layers, fc_neural, fc_block)
    model = model.cuda()
    optimizer = torch.optim.Adam(params = model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

    for epoch in range(1, max_epochs + 1):
        print('=' * 20, 'Epoch', epoch, '=' * 20)
        train_acc, train_loss = train(train_loader, model, criterion, optimizer, True)
        val_acc, val_loss = val(val_loader, model, criterion, True)

        train_acc_list.append(train_acc)
        train_loss_list.append(train_loss)
        val_acc_list.append(val_acc)
        val_loss_list.append(val_loss)

        print('Train Acc: {:.6f} Train Loss: {:.6f}'.format(train_acc, train_loss))
        print('  Val Acc: {:.6f}   Val Loss: {:.6f}'.format(val_acc, val_loss))
    return val_acc

In [ ]:
def comparison(parameter, lower_bound, upper_bound, integer = False, mutation_rate = 0.02):
    new_parameter = parameter
    if parameter > upper_bound:
        new_parameter =  np.random.uniform(low = (lower_bound+upper_bound)/2, high = upper_bound)
    elif parameter < lower_bound:
        new_parameter =  np.random.uniform(low = lower_bound, high = (lower_bound+upper_bound)/2)
    else:
        if random.uniform(0,1) < mutation_rate:
            new_parameter = np.random.uniform(low = lower_bound, high = upper_bound)
            print("Mutation occur:", parameter, "-->", new_parameter)
    if integer:
        new_parameter = int(new_parameter)
    return new_parameter

In [ ]:
def communication(i, iteration_per_communicate, local_number, receive_numbers, Gb, add, score_ind):
  write_name = str(i)+ str(local_number) +".csv"
  pd.DataFrame(Gb).to_csv(add+write_name)
  for receive_number in receive_numbers:
    patience = 0
    check = True
    while check:
      try:
        read_name = str(i)+ str(receive_number) +".csv"
        Gb_from_others = pd.read_csv(add+read_name)
        Gb_from_others = Gb_from_others.to_numpy().transpose()[1,]  
        print("receive from", receive_number, Gb_from_others)
        if Gb_from_others[score_ind] > Gb[score_ind]:
          Gb = Gb_from_others

        check = False
      except:
        time.sleep(1)
        patience += 1
        if patience > 4*60:
          print("break!!!!!! I loss my patience")
          break
  return Gb

In [ ]:
p_num = 4  ##粒子個數
max_iteration = 10 ##迭代次數
score_ind = 11 ## 分數的 index
record = np.zeros((1, max_iteration), dtype='float')
boundary = [[1e-5, 5e-3],[1,64],[1,8],[1,16],[1,30],[1,16],[4,4],[1, 50], [1, 5], [32,256], [1,3]]
x = np.array([[1.5e-4, 59,  3,1,1,1,  4, 30, 1, 256, 1, 0.398750],
              [  5e-4, 40,  3,6,5,3,  4, 30, 1, 128, 2, 0.426250],
              [  1e-4, 48,  4,3,2,1,  4, 31, 2,  64, 3, 0.261250],
              [  5e-3, 20,  6,6,16,3, 4, 20, 2,  32, 1, 0.373750],
             ])
x = x.transpose()
for j in range(p_num):
    for k in range(score_ind):
        if k != 0:
            x[k,j] = comparison(x[k,j], boundary[k][0], boundary[k][1], True,  mutation_rate = 1)
    gc.collect()
    with torch.no_grad():
        torch.cuda.empty_cache()
    time.sleep(10)
    x[score_ind,j] = RESTM_Train(x[:score_ind,j].tolist())
          
# x = np.array([[1.5000e-04, 5.0000e-04, 1.0000e-04, 5.0000e-03],
#  [4.2000e+01, 1.6000e+01, 5.4000e+01, 1.0000e+00],
#  [4.0000e+00, 5.0000e+00, 4.0000e+00, 2.0000e+00],
#  [1.1000e+01, 1.1000e+01, 7.0000e+00, 1.3000e+01],
#  [1.8000e+01, 4.0000e+00, 4.0000e+00, 1.2000e+01],
#  [2.0000e+00, 3.0000e+00, 1.0000e+01, 1.0000e+00],
#  [4.0000e+00, 4.0000e+00, 4.0000e+00, 4.0000e+00],
#  [1.1000e+01, 3.0000e+00, 8.0000e+00, 2.5000e+01],
#  [3.0000e+00, 1.0000e+00, 3.0000e+00, 3.0000e+00],
#  [1.7400e+02, 1.8700e+02, 2.2000e+02, 1.2000e+02],
#  [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00],
#  [2.3250e-01, 2.1375e-01, 1.0125e-01, 3.1375e-01]])

print(x)

[[1.5000e-04 5.0000e-04 1.0000e-04 5.0000e-03]
 [4.2000e+01 1.6000e+01 5.4000e+01 1.0000e+00]
 [4.0000e+00 5.0000e+00 4.0000e+00 2.0000e+00]
 [1.1000e+01 1.1000e+01 7.0000e+00 1.3000e+01]
 [1.8000e+01 4.0000e+00 4.0000e+00 1.2000e+01]
 [2.0000e+00 3.0000e+00 1.0000e+01 1.0000e+00]
 [4.0000e+00 4.0000e+00 4.0000e+00 4.0000e+00]
 [1.1000e+01 3.0000e+00 8.0000e+00 2.5000e+01]
 [3.0000e+00 1.0000e+00 3.0000e+00 3.0000e+00]
 [1.7400e+02 1.8700e+02 2.2000e+02 1.2000e+02]
 [1.0000e+00 1.0000e+00 1.0000e+00 1.0000e+00]
 [2.3250e-01 2.1375e-01 1.0125e-01 3.1375e-01]]


In [ ]:
w = 0.5
c1 = 2
c2 = 2
V = np.zeros((score_ind, p_num))
Pb = x.copy()
Gb = Pb[:, np.argmax(Pb[score_ind])].copy()
Gb = communication(0, iteration_per_communicate, local_number, receive_numbers, Gb, add, score_ind)
print(Gb)

receive from 2 [5.0000e-04 1.9000e+01 7.0000e+00 3.0000e+00 2.3000e+01 4.0000e+00
 4.0000e+00 3.4000e+01 4.0000e+00 2.5500e+02 1.0000e+00 2.9625e-01]
receive from 3 [5.0000e-04 5.3000e+01 4.0000e+00 2.0000e+00 1.8000e+01 9.0000e+00
 4.0000e+00 2.6000e+01 1.0000e+00 8.0000e+01 1.0000e+00 3.0375e-01]
receive from 4 [5.00e-03 3.50e+01 6.00e+00 1.10e+01 1.80e+01 8.00e+00 4.00e+00 2.10e+01
 3.00e+00 2.08e+02 1.00e+00 2.15e-01]
[5.0000e-03 1.0000e+00 2.0000e+00 1.3000e+01 1.2000e+01 1.0000e+00
 4.0000e+00 2.5000e+01 3.0000e+00 1.2000e+02 1.0000e+00 3.1375e-01]


In [ ]:
for i in range(1,(max_iteration+1)):
    print('-' * 20, 'Iteration', i, '-' * 20)
    for j in range(p_num):
        print('.' * 20, 'Particle', j+1, '.' * 20)
        for k in range(score_ind):
            V[k,j] = w * V[k,j] + c1 * random.uniform(0,1) * (Pb[k,j] - x[k,j]) + c2 * random.uniform(0,1) * (Gb[k] - x[k,j])  ##計算速度
            x[k,j] += V[k,j]  ##更新位置
            x[k,j] = comparison(x[k,j], boundary[k][0], boundary[k][1], (k != 0))
        gc.collect()
        with torch.no_grad():
            torch.cuda.empty_cache()
        time.sleep(10)
        x[score_ind,j] = RESTM_Train(x[:score_ind,j].tolist())
    for j in range(p_num): ##每個粒子的最好位置更新
        if Pb[score_ind,j] < x[score_ind,j]:
            Pb[:,j] = x[:,j].copy()
            print(Gb[score_ind], max(Pb[score_ind]))
    if Gb[score_ind] < max(Pb[score_ind]): 
        Gb = Pb[:, np.argmax(Pb[score_ind])].copy()

    if i % iteration_per_communicate == 0:
        Gb = communication(i, iteration_per_communicate, local_number, receive_numbers, Gb, add, score_ind)
    print(i,Gb)

-------------------- Iteration 1 --------------------
.................... Particle 1 ....................
 learning_rate = 0.0028187847549780024 initial_size = 5 
 blocks_1 = 3 blocks_2 = 14 blocks_3 = 17 blocks_4 = 1 
 expansion = 4 hidden_size = 29 num_layers = 3


  0%|          | 0/200 [00:00<?, ?it/s]

==================== Epoch 1 ====================


  0%|          | 0/200 [00:00<?, ?it/s]

Train Acc: 0.126212 Train Loss: 2.153193
  Val Acc: 0.146250   Val Loss: 2.068887
==================== Epoch 2 ====================


...

Validation: 100%|██████████| 25/25 [00:05<00:00,  4.36it/s]


Train Acc: 0.323428 Train Loss: 1.809141
  Val Acc: 0.328750   Val Loss: 1.756988
.................... Particle 3 ....................
 learning_rate = 0.00026302786117824665 initial_size = 53 
 blocks_1 = 1 blocks_2 = 11 blocks_3 = 5 blocks_4 = 2 
 expansion = 4 hidden_size = 25 num_layers = 1


  0%|          | 0/200 [00:00<?, ?it/s]

==================== Epoch 1 ====================


Training:  82%|████████▏ | 163/200 [01:39<00:21,  1.75it/s]